In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.special, scipy.integrate, scipy.signal, scipy.misc
from tqdm.notebook import tqdm as tqdm
import warnings
import h5py, sys

In [ ]:
M = 40
NA = 0.95
MLPitch = 150e-6
Nnum = 15
OSR = 1#???????? WHAT IS OSR? Probably 'oversampling ratio'? Doesn't appear in the PSF filename
       #str2num(settingPSF.OSR);
n = 1.0
fml = 3000e-6
lam = 520e-9;
zmax = 0
zmin = -13e-6
zspacing = 0.5e-6
warnings.warn('Altered so that we only process one z plane')
zmin = 0

In [ ]:
eqtol = 1e-10;

k = 2*np.pi*n/lam
k0 = 2*np.pi*1/lam
d = fml
ftl = 200e-3          #focal length of tube lens
fobj = ftl/M          # focal length of objective lens
fnum_obj = M/(2*NA)   # f-number of objective lens (imaging-side)
fnum_ml = fml/MLPitch # f-number of microlens

assert((Nnum%2)==1), 'Nnum should be an odd number'

In [ ]:
# JT: Load a .mat file generated by the actual Matlab code
def LoadRawMatrixData(matPath):
    # Load the matrices from the .mat file.
    # This is slow since they must be decompressed and are rather large! (9.5GB each, in single-precision FP)
    hReducedShape = []
    htReducedShape = []
    with h5py.File(matPath, 'r') as f:
        print('Load CAindex')
        sys.stdout.flush()
        _CAindex = f['CAindex'].value.astype('int')
        
        print('Load H')
        sys.stdout.flush()
        _H = f['H'].value.astype('float32')
        Nnum = _H.shape[2]
        aabbRange = int((Nnum+1)/2)        
        for cc in range(_H.shape[0]):
            HCC =  _H[cc, :aabbRange, :aabbRange, _CAindex[0,cc]-1:_CAindex[1,cc], _CAindex[0,cc]-1:_CAindex[1,cc]]
            hReducedShape.append(HCC.shape)

        print('Load Ht')
        sys.stdout.flush()
        _Ht = f['Ht'].value.astype('float32')

        for cc in range(_Ht.shape[0]):
            HtCC =  _Ht[cc, :aabbRange, :aabbRange, _CAindex[0,cc]-1:_CAindex[1,cc], _CAindex[0,cc]-1:_CAindex[1,cc]]
            htReducedShape.append(HtCC.shape)
        
    return (_H, _Ht, hReducedShape, htReducedShape, _CAindex)

(_H, _Ht, hReducedShape, htReducedShape, _CAindex) = LoadRawMatrixData('/Development/light-field-flow/PSFmatrix/PSFmatrix_M40NA0.95MLPitch150fml3000from-13to0zspacing0.5Nnum15lambda520n1.0.mat')

In [ ]:
def calcPSFFT(p3, fobj, NA, x1space, scale, lam, fml, M, n):
    k = 2*np.pi*n/lam
    alpha = np.arcsin(NA/n)
    p1 = 0
    p2 = 0

    psfLine = np.zeros((len(x1space)))
    for a in tqdm(range(len(x1space))):
        x1 = x1space[a]
        x2 = 0
        xL2normsq = (((x1+M*p1)**2+(x2+M*p2)**2)**0.5)/M
        v = k*xL2normsq*np.sin(alpha)   
        u = 4*k*p3*(np.sin(alpha/2)**2)

        Koi = M/((fobj*lam)**2)*np.exp(-1j*u/(4*(np.sin(alpha/2)**2)))
#        integrand = @(theta) (sqrt(cos(theta))) .* (1+cos(theta))  .*  (exp(-(i*u/2)* (sin(theta/2).^2) / (sin(alpha/2)^2)))  .*  (besselj(0, sin(theta)/sin(alpha)*v))  .*  (sin(theta));
        integrand = lambda theta: (np.sqrt(np.cos(theta))) * (1+np.cos(theta))  \
                                    *  (np.exp(-(1j*u/2)* (np.sin(theta/2)**2) / (np.sin(alpha/2)**2))) \
                                    *  (scipy.special.jn(0, np.sin(theta)/np.sin(alpha)*v)) \
                                    *  (np.sin(theta))
#        I0 = integral(@(theta)integrand (theta),0,alpha);  
        integrand_r = lambda theta: np.real(integrand(theta))
        integrand_i = lambda theta: np.imag(integrand(theta))
        # JT: I tried the additional precision (as commented out, below), but it did not
        # make any noticable difference to the outcome. i.e. the jagginess seems to be genuine.
        I0_r,err_r = scipy.integrate.quad(integrand_r, 0, alpha, limit=80)#, epsabs=1e-10, epsrel=1e-10)
        I0_i,err_i = scipy.integrate.quad(integrand_i, 0, alpha, limit=80)#, epsabs=1e-10, epsrel=1e-10)
        I0 = (I0_r + 1j*I0_i)
        err = (err_r + 1j*err_i)
        psfLine[a] =  np.abs((Koi*I0)**2)
    psfLine = psfLine / np.max(psfLine)
    return psfLine

In [ ]:
pixelPitch = MLPitch/Nnum # pitch of virtual pixels

# JT: not sure what this is for - maybe a feature that they never implemented?
x1objspace = np.array([0])
x2objspace = np.array([0])
x3objspace = np.arange(zmin, zmax+0.1*zspacing, zspacing)
objspace = np.ones((len(x1objspace),len(x2objspace),len(x3objspace)))


p3max = np.max(np.abs(x3objspace))
x1testspace = (pixelPitch/OSR) * np.arange(0,Nnum*OSR*20+1)
x2testspace = [0]   
psfLine = calcPSFFT(p3max, fobj, NA, x1testspace, pixelPitch/OSR, lam, d, M, n)

In [ ]:
outArea = np.where(psfLine<0.04)
if len(outArea[0]) == 0:
    raise('Estimated PSF size exceeds the limit');   
IMGSIZE_REF = int(np.ceil(outArea[0][0]/(OSR*Nnum)))

In [ ]:
# JT: Assuming I have translated this code correctly, it yields a rather off curve.
# It's a wiggly line superimposed on a broadly-decreasing function.
# I don't know if this is how it should be, or if there is an issue with my code translation.
# I will probably have to compare against the MATLAB.

#psfLineDefault = psfLine.copy()
plt.xlim(0,100)
plt.plot(psfLine)
plt.plot(psfLineDefault)

In [ ]:
def calcML(fml, k, x1MLspace, x2MLspace, x1space, x2space):
    x1length = len(x1space)
    x2length = len(x2space)
    x1MLdist = len(x1MLspace)
    x2MLdist = len(x2MLspace)
    # JT: this is a very strange code construction, but its aim appears to be to identify
    # one (any) index in x1space that is ==0, and take that as one 'center' in x1.
    # It then constructs a list of indices that represents *all* the 'centers' in x1.
    # The code relies on the fact that (x1center: -x1MLdist:1) is defined in matlab to use 
    # only the value of the first element of the array x1center when generating the range.
    #x1center = find(x1space==0);
    #x1centerALL = [  (x1center: -x1MLdist:1)  (x1center + x1MLdist: x1MLdist :x1length)];
    #x1centerALL = sort(x1centerALL);
    x1center = np.where(x1space==0)[0][0]
    x2center = np.where(x2space==0)[0][0]
    x1centerALL = np.append(np.arange(x1center, -1, -x1MLdist), \
                            np.arange(x1center+x1MLdist, x1length, x1MLdist))
    x2centerALL = np.append(np.arange(x2center, -1, -x2MLdist), \
                            np.arange(x2center+x2MLdist, x2length, x2MLdist))
    np.sort(x1centerALL)
    np.sort(x2centerALL)

    zeroline = np.zeros(len(x2space))

    patternML = np.zeros((len(x1MLspace), len(x2MLspace)), dtype='complex128')
    patternMLcp = np.zeros((len(x1MLspace), len(x2MLspace)), dtype='complex128')
    for a in range(len(x1MLspace)):
        for b in range(len(x2MLspace)):
            x1 = x1MLspace[a]
            x2 = x2MLspace[b]
            xL2norm = x1**2 + x2**2

            patternML[a,b] = np.exp(-1j*k/(2*fml)*xL2norm)
            patternMLcp[a,b] = np.exp(-0.05*1j*k/(2*fml)*xL2norm) 

    MLspace = np.zeros((len(x1space), len(x2space)))
    MLcenters = MLspace
    for a in range(len(x1centerALL)):
        for b in range(len(x2centerALL)):
            MLcenters[x1centerALL[a], x2centerALL[b]] = 1
    # JT TODO: I need to check whether fftconvolve is identical to conv2.
    # I seem to remember it might be different for odd-sized arrays or something?
    MLARRAY = scipy.signal.fftconvolve(MLcenters, patternML, 'same')
    MLARRAYcp = scipy.signal.fftconvolve(MLcenters, patternMLcp, 'same')

    # JT TODO: what is the purpose of these, then, given that they don't seem to be returned anywhere!?
    MLARRAYcpANG = np.angle(MLARRAYcp);
    MLARRAYcpANG = MLARRAYcpANG - np.min(MLARRAYcpANG) + 0.0
    MLARRAYcpANGnorm = MLARRAYcpANG/np.max(MLARRAYcpANG)
    return MLARRAY

In [ ]:
def calcPSF(p1, p2, p3, fobj, NA, x1space, x2space, scale, lam, MLARRAY, fml, M, n, centerArea):
    k = 2*np.pi*n/lam
    alpha = np.arcsin(NA/n)
    x1length = len(x1space)
    x2length = len(x2space)
    zeroline = np.zeros(len(x2space), dtype='complex128')

    pattern = np.zeros((x1length, x2length), dtype='complex128')
    centerPT = int(np.ceil(len(x1space)/2))
    for a in tqdm(range(centerArea[0],centerPT)):
        patternLine = zeroline.copy()
        for b in tqdm(range(a,centerPT), leave=False):
            x1 = x1space[a]
            x2 = x2space[b]
            xL2normsq = (((x1+M*p1)**2+(x2+M*p2)**2)**0.5)/M

            v = k*xL2normsq*np.sin(alpha)
            u = 4*k*(p3*1)*(np.sin(alpha/2)**2)
            Koi = M/((fobj*lam)**2)*np.exp(-1j*u/(4*(np.sin(alpha/2)**2)))
            #intgrand = @(theta) (sqrt(cos(theta))) .* (1+cos(theta))  .*  (exp(-(i*u/2)* (sin(theta/2).^2) / (sin(alpha/2)^2)))  .*  (besselj(0, sin(theta)/sin(alpha)*v))  .*  (sin(theta));
            #I0 = integral(@(theta)intgrand (theta),0,alpha);  
            integrand = lambda theta: (np.sqrt(np.cos(theta))) * (1+np.cos(theta))  \
                                    *  (np.exp(-(1j*u/2)* (np.sin(theta/2)**2) / (np.sin(alpha/2)**2))) \
                                    *  (scipy.special.jn(0, np.sin(theta)/np.sin(alpha)*v)) \
                                    *  (np.sin(theta))
            integrand_r = lambda theta: np.real(integrand(theta))
            integrand_i = lambda theta: np.imag(integrand(theta))
            I0_r,err_r = scipy.integrate.quad(integrand_r, 0, alpha, limit=80)
            I0_i,err_i = scipy.integrate.quad(integrand_i, 0, alpha, limit=80)
            I0 = (I0_r + 1j*I0_i)
            err = (err_r + 1j*err_i)
            patternLine[b] = Koi*I0
        pattern[a,:] = patternLine

    patternA = pattern[0:centerPT, 0:centerPT];
    patternAt = np.fliplr(patternA)

    pattern3D = np.zeros((pattern.shape[0], pattern.shape[1], 4), dtype='complex128');
    pattern3D[:,:,0] = pattern;
    pattern3D[0:centerPT, centerPT-1:end,0] = patternAt
    # JT: I have not considered whether this rotates in the same direction as python,
    # but it doesnt matter because we consider all four rotations and take the maximum!
    pattern3D[:,:,1] = np.rot90( pattern3D[:,:,0] , -1)
    pattern3D[:,:,2] = np.rot90( pattern3D[:,:,0] , -2)
    pattern3D[:,:,3] = np.rot90( pattern3D[:,:,0] , -3)

    pattern = np.maximum(pattern3D,axis=2)
#    %%%%%%%%%%%%%%%%%%% CALCULATED LF PSF %%%%%%%%%%%%%%%%%%%%%%%%%%%
    f1,dx1,x1 = fresnel2D(pattern*MLARRAY, scale, 1*fml, lam)

    return pattern, f1

def fresnel2D(f0,dx0,z,lam):
    (Nx,Ny) = f0.shape
    k = 2*np.pi/lam

    du = 1/(Nx*dx0)
    u = np.append(np.arange(0,np.ceil(Nx/2)), np.arange(np.ceil(-Nx/2),0))*du
    dv = 1/(Ny*dx0)
    v = np.append(np.arange(0,np.ceil(Ny/2)), np.arange(np.ceil(-Ny/2),0))*dv

    H = np.exp(-1j*2*np.pi**2 * (np.tile(u[:,np.newaxis],(1,len(v)))**2+np.tile(v,(len(u),1))**2)*z/k)
    f1 = np.exp(1j*k*z)*np.fft.ifft2(np.fft.fft2(f0) * H )
    dx1 = dx0
    x1 = np.arange(-Nx/2,Nx/2)*dx1
    return f1,dx1,x1                              

In [ ]:
def im_shift2(img, SHIFTX, SHIFTY):
    eqtol = 1e-10
    assert (np.abs(SHIFTX%1)<eqtol and np.abs(SHIFTY%1)<eqtol), 'SHIFTX and SHIFTY should be integer numbers'

    SHIFTX = int(round(SHIFTX))
    SHIFTY = int(round(SHIFTY))
    new_im = np.zeros(img.shape);

    # JT: logic for here: 0:end-SHIFTX in matlab would skip final element if SHIFTX=-1
    # In python, :-1 would skip final element too, so :-SHIFTX will do the trick.
    # However, care is needed to cope with SHIFTX=0, hence the use of endx
    endx,endy = img.shape
    if SHIFTX >=0 and SHIFTY >= 0:
        new_im[SHIFTX:, SHIFTY:] = img[:endx-SHIFTX, :endy-SHIFTY]
    elif SHIFTX >=0 and SHIFTY < 0:
        new_im[SHIFTX:, :SHIFTY] = img[:endx-SHIFTX, -SHIFTY:]
    elif SHIFTX <0 and SHIFTY >= 0:
        new_im[:SHIFTX, SHIFTY:] = img[-SHIFTX:, :endy-SHIFTY]
    else:
        new_im[:SHIFTX, :SHIFTY] = img[-SHIFTX:, -SHIFTY:]

    return new_im

In [ ]:
def pixelBinning(SIMG, OSR):
    x1length, x2length = SIMG.shape

    x1center = int((x1length-1)/2 + 1)
    x2center = int((x2length-1)/2 + 1)
    x1centerinit = x1center - int((OSR-1)/2)
    x2centerinit = x2center - int((OSR-1)/2)
    x1init = x1centerinit -  int(np.floor(x1centerinit/OSR)*OSR)
    x2init = x2centerinit -  int(np.floor(x2centerinit/OSR)*OSR)

    x1shift = 0
    x2shift = 0
    if x1init<1:
        x1init = x1init + OSR
        x1shift = 1
    if x2init<1:
        x2init = x2init + OSR
        x2shift = 1

    # JT: commented out in MATLAB code:  SIMG_crop = SIMG( (x1init:1:end-OSR+1), (x2init:1:end-OSR+1) );
    # JT: commented out in MATLAB code:  SIMG_crop = SIMG_crop( (1:1: floor(size(SIMG_crop,1)/OSR)*OSR) ,  (1:1: floor(size(SIMG_crop,2)/OSR)*OSR) );
    halfWidth = len(range(x1init,x1center))
#    SIMG_crop = SIMG( [ (x1init:x1center-1) x1center x1center+1:x1center+halfWidth ],  [ (x2init:x2center-1) x2center x2center+1:x2center+halfWidth ] );
    SIMG_crop = SIMG[ x1init-1:x1center+halfWidth,  
                      x2init-1:x2center+halfWidth]

#    %%%%%%%%%%%%%%%%%% PIXEL BINNING  %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
    # JT: I am not totally certain I am doing the same as the matlab,
    # but this achieves what I think I would expect the matlab to do!
    m,n = SIMG_crop.shape 
    SIMG_crop = np.reshape(SIMG_crop, (int(m/OSR), OSR, int(n/OSR), OSR))
    OIMG = np.sum(SIMG_crop, axis=(1,3))
    #SIMG_crop = sum( reshape(SIMG_crop,OSR,[]) ,1 );
    #SIMG_crop=reshape(SIMG_crop,m/OSR,[]).'; %Note transpose
    #SIMG_crop=sum( reshape(SIMG_crop,OSR,[]) ,1);
    #OIMG =reshape(SIMG_crop,n/OSR,[]).'; %Note transpose

    return OIMG, x1shift, x2shift

In [ ]:
print('Size of PSF ~= {0} [microlens pitch]'.format(IMGSIZE_REF))
IMG_HALFWIDTH = np.maximum(Nnum*(IMGSIZE_REF + 1), 2*Nnum)
print('Size of IMAGE = {0}x{1}'.format(IMG_HALFWIDTH*2*OSR+1, IMG_HALFWIDTH*2*OSR+1))
x1space = (pixelPitch/OSR)*np.arange(-IMG_HALFWIDTH*OSR, IMG_HALFWIDTH*OSR+0.1, 1); 
x2space = (pixelPitch/OSR)*np.arange(-IMG_HALFWIDTH*OSR, IMG_HALFWIDTH*OSR+0.1, 1); 
x1length = len(x1space)
x2length = len(x2space)

x1MLspace = (pixelPitch/OSR)* np.arange(-(Nnum*OSR-1)/2 , (Nnum*OSR-1)/2+0.1, 1)
x2MLspace = (pixelPitch/OSR)* np.arange(-(Nnum*OSR-1)/2 , (Nnum*OSR-1)/2+0.1, 1)
x1MLdist = len(x1MLspace)
x2MLdist = len(x2MLspace)

#%%%%%%%%%%%%%%%%%% FIND NON-ZERO POINTS %%%%%%%%%%%%%%%%%%%%%%%%%%
validpts = np.where(objspace>eqtol)
numpts = len(validpts[0])
# Matlab code:
#  [p1indALL p2indALL p3indALL] = ind2sub( size(objspace), validpts);
#  p1ALL = x1objspace(p1indALL)';
(p1indALL, p2indALL, p3indALL) = validpts
p1ALL = x1objspace[p1indALL]
p2ALL = x2objspace[p2indALL]
p3ALL = x3objspace[p3indALL]

#%%%%%%%%%%%%%%%%%%%%%%%% DEFINE ML ARRAY %%%%%%%%%%%%%%%%%%%%%%%%% 
MLARRAY = calcML(fml, k0, x1MLspace, x2MLspace, x1space, x2space)

#%%%%%%%%%%%%%%%%%%%%%% Alocate Memory for storing PSFs %%%%%%%%%%%   
LFpsfWAVE_STACK = np.zeros((x1length, x2length, numpts))
psfWAVE_STACK = np.zeros((x1length, x2length, numpts))
print('Start Calculating PSF...')

# Part 1: "Projection from single point"

In [ ]:
centerPT = int(np.ceil(len(x1space)/2))
halfWidth =  Nnum*(IMGSIZE_REF + 0 )*OSR
centerArea = np.arange(np.maximum((centerPT - halfWidth),1)-1, 
                       np.minimum((centerPT + halfWidth),len(x1space)-1))

print('Computing PSFs (1/3)')
for eachpt in range(numpts):
    p1 = p1ALL[eachpt]
    p2 = p2ALL[eachpt]
    p3 = p3ALL[eachpt]
    
    IMGSIZE_REF_IL = np.ceil(IMGSIZE_REF*( np.abs(p3)/p3max))
    halfWidth_IL =  np.maximum(Nnum*(IMGSIZE_REF_IL + 0 )*OSR, 2*Nnum*OSR)
    centerArea_IL = np.arange(np.maximum((centerPT - halfWidth_IL),1)-1,
                              np.minimum((centerPT + halfWidth_IL),len(x1space)), dtype=np.int)
    print('{0} size of center area = {1}x{2}'.format(eachpt, len(centerArea_IL), len(centerArea_IL)))
    
    # excute PSF computing function
    if False:
        psfWAVE, LFpsfWAVE = calcPSF(p1, p2, p3, fobj, NA, x1space, x2space, pixelPitch/OSR, lam, MLARRAY, d, M, n,  centerArea_IL)
        psfWAVE_STACK[:,:,eachpt]  = psfWAVE
        LFpsfWAVE_STACK[:,:,eachpt]= LFpsfWAVE
    else:
        warnings.warn('Not actually computing PSF!')

    

# Part 2: Compute light field PSFs

In [ ]:
x1objspace = (pixelPitch/M)*np.arange(-np.floor(Nnum/2), np.floor(Nnum/2)+0.1, 1)
x2objspace = x1objspace.copy()
XREF = np.ceil(len(x1objspace)/2)
YREF = np.ceil(len(x1objspace)/2)
CP = np.arange((centerPT-1)/OSR+1 - halfWidth/OSR, (centerPT-1)/OSR+1 + halfWidth/OSR +0.1, 1, dtype=np.int)
H = np.zeros((len(CP), len(CP), len(x1objspace), len(x2objspace), len(x3objspace) ))

print('Computing LF PSFs (2/3)')
if False:
    for i in tqdm(range(len(x1objspace)*len(x2objspace)*len(x3objspace))):
        (a, b, c) = np.unravel_index(i, (len(x1objspace), len(x2objspace), len(x3objspace)))
        psfREF = psfWAVE_STACK[:,:,c]

        psfSHIFT = im_shift2(psfREF, OSR*(a-XREF), OSR*(b-YREF) )
        f1,dx1,x1 = fresnel2D(psfSHIFT*MLARRAY, pixelPitch/OSR, d,lam)
        f1 = im_shift2(f1, -OSR*(a-XREF), -OSR*(b-YREF) )

        xmin =  np.maximum( centerPT  - halfWidth, 0)

        xmax =  np.minimum( centerPT  + halfWidth, f1.shape[0]-1)
        ymin =  np.maximum( centerPT  - halfWidth, 0)
        ymax =  np.minimum( centerPT  + halfWidth, f1.shape[1]-1)

        f1_AP = np.zeros(f1.shape)
        f1_AP[xmin:xmax,ymin:ymax] = f1[xmin:xmax,ymin:ymax]
        [f1_AP_resize, x1shift, x2shift] = pixelBinning(np.abs(f1_AP**2), OSR)      
        f1_CP = f1_AP_resize[ CP - x1shift, CP-x2shift ]

        H[:,:,a,b,c] = f1_CP
    H = H/np.max(H)
    
    # Free up memory from variables we have now finished with
    del f1
    del f1_AP
    del f1_AP_resize
    del f1_CP
    del psfREF
    del psfSHIFT
    del LFpsfWAVE_STACK
    del psfWAVE_STACK
else:
    warnings.warn('Skipping LF PSF calculation')
    H = _H.T.copy()

x1space = (pixelPitch/1)*np.arange(-IMG_HALFWIDTH*1, IMG_HALFWIDTH*1+0.1, 1);
x2space = x1space.copy()
x1space = x1space[CP]
x2space = x2space[CP]

if False:
    tol = 0.005
    for i in tqdm(range(H.shape[4]), desc='clipping to zero'):
        H4Dslice = H[:,:,:,:,i]
        H4Dslice[np.where( H4Dslice < (tol*np.max(H4Dslice)) )] = 0
        H[:,:,:,:,i] = H4Dslice
else:
    warnings.warn('Not clipping to zero')

# Part 3: calculate Ht
This appears to be correct now. If I run this code here, starting with H loaded from a real Matlab .mat file, the result I get for Ht (or, at least, for certain PSFs selected from it) is basically identical to the Ht stored in the original .mat file.

In [ ]:
def Rotate180(img):
    h = img.shape[0]
    rot_img = np.zeros_like(img)
    for i in range(h):
        rot_img[i] = img[h-i-1,::-1]
    return rot_img

def backwardProject(H, projection, Nnum, ccRange=None):
    x3length = H.shape[4]
    if (ccRange is None):
        ccRange = range(x3length)
    Backprojection = np.zeros((projection.shape[0], projection.shape[0], x3length))
    for aa in tqdm(range(Nnum), leave=False):
        for bb in tqdm(range(Nnum), leave=False):
            for cc in ccRange:
                Ht = Rotate180(H[:,:,aa,bb,cc])
                tempSlice = scipy.signal.fftconvolve(projection, Ht, 'same')
                Backprojection[aa::Nnum, bb::Nnum, cc] += tempSlice[aa::Nnum, bb::Nnum]
    return Backprojection

def calcHt(H):
    Hsize1,_,Nnum,_,x3length = H.shape
    tmpsize = int(np.ceil(H.shape[0]/Nnum))
    if ((tmpsize%2) == 1):
        imgsize = (tmpsize+2)*Nnum;
    else:
        imgsize = (tmpsize+3)*Nnum,

    zeroprojection = np.zeros((imgsize, imgsize))
    imcenter_m = int(np.ceil(imgsize/2))
    imcenterinit_m = imcenter_m - int(np.ceil(Nnum/2))

    Ht = np.zeros_like(H)
    for aa in tqdm(range(Nnum)):
        for bb in tqdm(range(Nnum), leave=False):
            temp = zeroprojection.copy()
            temp[imcenterinit_m+aa, imcenterinit_m+bb] = 1  #√
            tempback = backwardProject(H, temp, Nnum)
            tempback_cut = tempback[imcenter_m - int((Hsize1-1)/2) - 0*Nnum - 1 : imcenter_m + int((Hsize1-1)/2) + 0*Nnum, 
                                    imcenter_m - int((Hsize1-1)/2) - 0*Nnum - 1 : imcenter_m + int((Hsize1-1)/2) + 0*Nnum]#√
            tempback_shift = np.zeros_like(tempback_cut)
            for cc in range(x3length):
                temp = im_shift2(tempback_cut[:,:,cc], int(np.ceil(Nnum/2)-aa-1), int(np.ceil(Nnum/2)-bb-1) ) #√

                if False:
                    plt.title('tempback {0},{1},{2}'.format(aa,bb,cc))
                    plt.imshow(temp)
                    plt.colorbar()
                    plt.show()
                    plt.title('_Ht {0},{1},{2}'.format(aa,bb,cc))
                    plt.imshow(_Ht.T[:,:,aa,bb,cc])
                    plt.colorbar()
                    plt.show()
                    plt.title('diff')
                    plt.imshow((_Ht.T[:,:,aa,bb,cc]-temp))
                    plt.colorbar()
                    plt.show()
                Ht[:,:,aa,bb,cc] = temp
    return Ht

In [ ]:
#%%%%%%%%%%%% Calculate Ht (transpose for backprojection) %%%%%%%%%
print('Computing Transpose (3/3)')
if True:
    Ht = calcHt(H)
else:
    warnings.warn('Not computing transpose!')
    Ht = H.copy()

H = H.astype('float32')
Ht = Ht.astype('float32')

#%%%%%%%%%%%%%%%%% Estimate PSF size again  %%%%%%%%%%%%%%%%%%%%%%%
# JT: I do indeed want to save CAindex in 1-based MATLAB indexing.
# My python deconvolution code expects that and will adjust it accordingly.
centerCP = np.ceil(len(CP)/2)
CAindex = np.zeros((len(x3objspace),2))
for i in range(len(x3objspace)):
    IMGSIZE_REF_IL = np.ceil(IMGSIZE_REF*( np.abs(x3objspace[i])/p3max))
    halfWidth_IL =  np.maximum(Nnum*(IMGSIZE_REF_IL + 0 ), 2*Nnum)
    CAindex[i,0] = np.maximum( centerCP - halfWidth_IL , 1)
    CAindex[i,1] = np.minimum( centerCP + halfWidth_IL , H.shape[0])

#print('Saving PSF matrix file...')
#save([savePath fileName] , 'H','Ht', 'CAindex', 'settingPSF', 'OSR', 'fobj', 'd', 'NA', 'objspace', 'M', 'MLARRAY', 'zspacing','x1objspace', 'x2objspace', 'x3objspace', 'pixelPitch', 'x1space','x2space', 'CP','Nnum' ,'-v7.3');
print('PSF computation complete.')